## This code extracts labels from a zooniverse export and reformats it into the yolo format

In [30]:
import pandas as pd
import ast
import numpy as np
import re
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
import cv2
from collections import Counter
import shutil
from tqdm import tqdm
import filecmp
import ast

In [31]:
# re formats the coordanates for use with yolo v5
# In:
#      x2 (list) : x coordanates for all labels for a single image 
#      y2 (list) : y coordanates for all labels for a single image 
#      w2 (list) : width distances for all labels for a single image 
#      h2 (list) : height distances for all labels for a single image 
#      origW (list) : Original width of the target image
#      origH (list) : Original height of the target image
# Out: 
#      x3 (list) : x coordanates after yoloV5 preperation
#      y3 (list) : y coordanates after yoloV5 preperation
#      w3 (list) : Height distances after yoloV5 preperation
#      h3 (list) : height distances after yoloV5 preperation
def PrepForYolo (x,y,w,h,origW,origH):
    x2 = []
    y2 = []
    w2 = []
    h2 = []
    for q in range(len(x)):
        # moves anchor to the middle of the bounding box
        x2.append(x[q]+(w[q]/2))
        y2.append(y[q]+(h[q]/2))
        # normalises the values
        x2[q] = x2[q]/origW
        y2[q] = y2[q]/origH
        w2.append(w[q]/origW)
        h2.append(h[q]/origH)
    return(x2, y2, w2, h2)


# This function creates the dataset for all volunteers. It creates the relivant file structures.
# In:
#      fmat (dictionary) : formatted bounding box information in yolo format for a single image
#      ip (list) : the encrypted ip address of the annotators for each bounding box for a single image
#      path (str) :  location of the intended save path 
#      pathType (str) : IIC or all users 
#      saveName (str) : name of the current image
#      imgLoc (str) : location of the images 
# Out: 
#      None
def SaveCrowd (fmat, ip, path, pathType, saveName, imgLoc):
        
    np.savetxt(path+'\\'+pathType+'\\labels\\train\\'+saveName+'.txt', fmat.values, fmt='%i %1.10f %1.10f %1.10f %1.10f')
    np.savetxt(path+'\\'+pathType+'\\volunteers\\train\\'+saveName+'.txt', ip, fmt='%s')
    
    shutil.copyfile(imgLoc+'\\'+saveName+'.JPG' , path+'\\'+pathType+'\\images\\train\\'+saveName+'.JPG')
    
    # return(saveName+' Done')

# This function creates the IID dataset. It creates the relivant file structures.
# In:
#      fmat (list) : formatted bounding box information in yolo format for a single image and annotator
#      ip (str) : the encrypted ip address for a single annotators for a single image
#      path (str) :  location of the intended save path 
#      pathType (str) : IIC or all users 
#      saveName (str) : name of the current image
#      imgLoc (str) : location of the images 
# Out: 
#      None
def SaveIID (fmat, ip, path, pathType, saveName, imgLoc):
        
    np.savetxt(path+'\\'+pathType+'\\labels\\train\\'+saveName+'.'+ip+'.txt', fmat.values, fmt='%i %1.6f %1.6f %1.6f %1.6f')
    
    shutil.copyfile(imgLoc+'\\'+saveName+'.JPG' , path+'\\'+pathType+'\\images\\train\\'+saveName+'.'+ip+'.JPG')
    # return(None)

    
    
    
# This function handels the train val splitting
def valSplit(TTs, path, pathType, isIID):
    direc = os.listdir(path+'\\'+pathType+'\\images\\train')
    count = len(direc)
    valSplit = round(count * (1-TTs))
    for i in range(valSplit):
        name, extension= os.path.splitext(direc[i])

        shutil.move(path+'\\'+pathType+'\\images\\train\\'+direc[i] , path+'\\'+pathType+'\\images\\val\\'+direc[i])
        shutil.move(path+'\\'+pathType+'\\labels\\train\\'+name+'.txt' , path+'\\'+pathType+'\\labels\\val\\'+name+'.txt')
        if isIID == False:
            shutil.move(path+'\\'+pathType+'\\volunteers\\train\\'+name+'.txt' , path+'\\'+pathType+'\\volunteers\\val\\'+name+'.txt')
        
        
    
    
# This function gets the location of the test dataset, and copys it into the correct format
def injectTestDS(testLocImages, testLocLabels, trgtLoc, pathType, iid):
    direc = os.listdir(testLocImages)
    for i in direc:
        name, extension= os.path.splitext(i)
        shutil.copyfile(testLocImages+'\\'+i , trgtLoc+'\\'+pathType+'\\images\\test\\'+i)
        shutil.copyfile(testLocLabels+'\\'+name+'.txt' , trgtLoc+'\\'+pathType+'\\labels\\test\\'+name+'.txt')
        if iid == False:
            label = open(testLocLabels+'\\'+name+'.txt', "r")
            rows = len([f.strip() for f in label])
            expert = []
            for l in range(rows):
                expert.append('expert')
            np.savetxt(trgtLoc+'\\'+pathType+'\\volunteers\\test\\'+name+'.txt', expert, fmt='%s')
            
            

# This function removes repeated images in the train dataset that are already in the test dataset. So the test dataset is fully unseen
def removeRepeated(testLocImg, trainLoc, pathType):
    delImg=[]
    trainLoc2 = trainLoc+'\\'+pathType+'\\images\\train'
    direcTest = os.listdir(testLocImg)
    direcTrain = os.listdir(trainLoc2)
    for tst in direcTest:
        for trn in direcTrain:
            name, extension= os.path.splitext(tst)
            name2, extension2= os.path.splitext(trn)
            if name in name2:
                # Counts repeated images
                delImg.append(name2)
                # deletes image in the train dataset
                os.remove(trainLoc2 + '\\' + trn)
                # deletes label in the train dataset
                os.remove(trainLoc + '\\' + pathType + '\\labels\\train\\' + name2 + '.txt')
                # deletes volunteer information in the train dataset
                try:
                    os.remove(trainLoc + '\\' + pathType + '\\volunteers\\train\\' + name2 + '.txt')
                except:
                    pass
    print(str(len(delImg)) + ' Repeated Images. They Have Been Deleted: ')
    print(delImg)


# this function ensures each bounding box is within the bounds of the image
def fixOutOfBoundsBoxes(x, y, w, h):
    for pos in range(len(x)):
        # the bounding box extends past the right of the image
        if (x[pos]+(w[pos]/2))>1:
            w[pos] = w[pos]-((x[pos]+(w[pos]/2))-1)
            x[pos] = 1-(w[pos]/2)
        # the bounding box extends past the left of the image
        elif (x[pos]-(w[pos]/2))<0:
            w[pos] = w[pos]+(x[pos]-(w[pos]/2))
            x[pos] = w[pos]/2
        # the bounding box extends past the bottom of the image
        if (y[pos]+(h[pos]/2))>1:
            h[pos] = h[pos]-((y[pos]+(h[pos]/2))-1)
            y[pos] = 1-(h[pos]/2)
        # the bounding box extends past the top of the image
        elif (y[pos]-(h[pos]/2))<0:
            h[pos] = h[pos]+(y[pos]-(h[pos]/2))
            y[pos] = h[pos]/2
    return(x,y,w,h)

# switches 2 labels of the users choice
def switchLabel(swapLabel, label):
    if label["value"] == swapLabel[0]:
        label["value"] = swapLabel[1]
    elif label["value"] == swapLabel[1]:
        label["value"] = swapLabel[0]
    return(label)
        
# # tests in turn A(anchor before bounds), A(anchor past bounds), C(anchor before bounds), C(anchor past bounds), B(anchor before bounds), B(anchor past bounds), D(anchor before bounds), D(anchor past bounds)
# # then repeated for the corner examples (AB, CD, DA, BC) with anchor inside and outside of bounds
# X1, Y1, W1, H1 = fixOutOfBoundsBoxes([0.95,1.1,0.002,-0.065,0.5,0.5,0.5,0.5],[0.5,0.5,0.5,0.5,0.95,1.1,0.002,-0.065],[0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28],[0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24])
# X2, Y2, W2, H2 = fixOutOfBoundsBoxes([0.95,1.1,0.002,-0.065,0.95,1.1,0.002,-0.065],[0.95,1.1,0.002,-0.065,0.002,-0.065,0.95,1.1],[0.28,0.28,0.28,0.28,0.28,0.28,0.28,0.28],[0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24])
# print('test1 (A1,A2,C1,C2,B1,B2,D1,D2): ',X1,W1)
# print('test1 (AB1,AB2,CD1,CD2,DA1,DA2,BC1,BC2): ',X2,W2)

In [33]:
# this section extracts all images and labels from the zooniverse output csv


##### File Locations #####

# file path to zooniverse
dataset = pd.read_csv (r'C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\Labels\dental-disease-labelling-easy-classifications.csv')
# file path that contains the images
imageLocation = r'C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\Data'
# The location you want to save the output to
saveLoaction = r'C:\Users\rb01243\OneDrive - University of Surrey\Desktop\zoon data\Finished Data'
# location of the test image dataset labels (compleeted by a single expert 'ground truth labels')
testLocationImg = r'C:\Users\rb01243\OneDrive - University of Surrey\Documents\GitHub\miccai\miccai_dental_disease\data\datasets\master\images'
# location of the test label dataset labels (compleeted by a single expert 'ground truth labels')
testLocationLbl = r'C:\Users\rb01243\OneDrive - University of Surrey\Documents\GitHub\miccai\miccai_dental_disease\data\datasets\master\labels\Jonathan'


#expertTestData = r'test'

#########################

######## Params #########

# type of prefix dental experts use
dentPrefix = 'dnt_'     # 'dnt_'
# the split ratio for train and val datasets (ground truth Train data is already in the correct format) (between 0.0 and 1 foat)
trainValSplit = 0.80
# if you want only expert labels change this filter to True
filterExpert = False
# minimum number of annotators per image
minAnnotators = 3
# True if you want to create an IID dataset as well as a crowdsourced dataset (IID: crowdsourced labels for a single image are split up into their own .txt files, to act as repeated ground truth labels)
includeIID = True   # True
# if you want to generate a test dataset using the 
includeTestDS = True
# Parameter to choose the minimum number of classifications a volunteer has to complete
minClss = 4
# The minimum workflow version
minWorkflowVersion = 20.00
# user this to remove a label type from the dataset
remveLbl = 0 #       -1 = none     0 = Calculus Plaque,   1 = Dental Caries,      2 = Bone Loss        This parameter will also shift all subsequent labels down by 1 to compensate of the removal of the value
# swap label (done before label removal [remveLbl])
swapLabel = [1,2] #    Leave empty to not swap any labels      ZOONIVERSE EPORT IS IN THE FORMAT OF (0 = CALCULUS, 1 = CARIES, 2 = BONE LOSS) [0,2] = 3 label format, [1,2] = 2 label format
# specific images to remove
imgRemove = ['Unknown-X-20200928-091843-XIQ+LFYOKA+S-0-YunpengÔÇÖs iMac.JPG']
########################

# Name of the folder storing the images
pthType = 'All_Volunteers_Calc_Removed' # All_Volunteers   All_Volunteers_Calc_Removed_John_Test

# filters expert labels only
if filterExpert == True:
    pthType = 'Expert_Volunteers'
    tempDS = dataset
    dataset = tempDS.loc[(tempDS['expert'] == 'expert')|(tempDS['gold_standard'] == True)|(tempDS['user_name'].str.contains(dentPrefix))] # filters for experts only
    dataset.reset_index()
    

# removes banned users
fle = open("banned.txt", "r")
banedUsers = eval(fle.read())
for bnd in banedUsers:
    dataset = dataset[dataset['user_name'] != bnd]
    


dataset = dataset[dataset["workflow_version"] > minWorkflowVersion].reset_index(drop=True)# removes pre-beta test data

dataset = dataset[dataset["annotations"].str.contains('"value":null') == False].reset_index(drop=True) # removes null bounding boxes
dataset = dataset[dataset["annotations"].str.contains('"value":\[]') == False].reset_index(drop=True) # removes empty classifications

dataset = dataset[dataset.groupby('user_name').user_name.transform('count') >= minClss].reset_index(drop=True) # removes users with less total classifications than the minClss variable




# Creates Directories
pathType = pthType+'_Crowdsourced'
try:
    os.mkdir(saveLoaction+'\\'+pathType)
except:
    print('Root Already Created')
try:
    os.mkdir(saveLoaction+'\\'+pathType+'\\images')
    os.mkdir(saveLoaction+'\\'+pathType+'\\images\\train')
    os.mkdir(saveLoaction+'\\'+pathType+'\\images\\test')
    os.mkdir(saveLoaction+'\\'+pathType+'\\images\\val')
except:
    print('Images Folder Already Created')
try:
    os.mkdir(saveLoaction+'\\'+pathType+'\\labels')
    os.mkdir(saveLoaction+'\\'+pathType+'\\labels\\train')
    os.mkdir(saveLoaction+'\\'+pathType+'\\labels\\test')
    os.mkdir(saveLoaction+'\\'+pathType+'\\labels\\val')
except:
    print('Labels Folder Already Created')
try:
    os.mkdir(saveLoaction+'\\'+pathType+'\\volunteers')
    os.mkdir(saveLoaction+'\\'+pathType+'\\volunteers\\train')
    os.mkdir(saveLoaction+'\\'+pathType+'\\volunteers\\test')
    os.mkdir(saveLoaction+'\\'+pathType+'\\volunteers\\val')
except:
    print('Volunteers Folder Already Created')


if includeIID == True:
    pathTypeIID = pthType+'_IID'
    try:
        os.mkdir(saveLoaction+'\\'+pathTypeIID)
    except:
        print('IID Root Already Created')
    try:
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\images')
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\images\\train')
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\images\\test')
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\images\\val')
    except:
        print('Images Folder Already Created')
    try:
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\labels')
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\labels\\train')
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\labels\\test')
        os.mkdir(saveLoaction+'\\'+pathTypeIID+'\\labels\\val')
    except:
        print('Labels Folder Already Created')




    
indexDone = []
#first volunteer for one image
for index, row in tqdm(dataset.iterrows()):
    width, height, xCord, yCord, labels, IP = [], [], [], [], [], []
    if index not in indexDone:
        # Removes Unwanted Images
        remove = eval(row['subject_data'].replace("null", "'null'"))
        remove = remove[str(row['subject_ids'])]
        remove = remove['Filename'].replace(".jpg", ".JPG")
        if remove in imgRemove:
            indexDone.append(index)
            print('REMOVED UNWANTED IMAGES: ', remove)
            continue
        temp = dataset.loc[dataset['subject_ids'] == (row['subject_ids'])]
        temp = temp.reset_index(drop=True)
        numAnno = len(temp)        
        # removes annotations and images if there aren't enough annotatiors (see parameters)
        if numAnno < minAnnotators:
            continue
        # all other volunteers for one image
        for index2, row2, in temp.iterrows():
            
            # extracts the username (encrypted IP address)
            userIP = row2['user_name']
            
            # extracts the name of the image
            imageName = row2['subject_data']
            imageName = imageName.replace("null", "'null'")
            imageName = eval(imageName)
            imageName = imageName[str(row2["subject_ids"])]
            imageName = imageName['Filename']
            name, extension= os.path.splitext(imageName)
            
            # fixes file extension for consistency
            if extension == '.jpg':
                imageName.replace(".jpg", ".JPG")
                imageName = (name+'.JPG')
            
            # extracts the annotation made by the user (encrypted IP address)
            annotation = row2['annotations']
            annotation = annotation.replace("null", "'null'")
            annotation = eval(annotation)
            annotation = annotation[0]["value"]
            
            #extracts the image width and height
            try:
                imgSze = row2['metadata']
                imgSze = imgSze.replace("true", "True")
                imgSze = imgSze.replace("null", "'null'")
                imgSze = eval(imgSze.replace("false", "False"))
                imgSze = imgSze['subject_dimensions'][0]
                originalwidth, originalheight = imgSze['naturalWidth'], imgSze['naturalHeight']
            except:
                pass
            
            # traverses the annotation dictionary inside the dataframe to extract all annotations.
            widthIID, heightIID, xCordIID, yCordIID, labelsIID = [], [], [], [], []
            for ann in annotation:
                label = ann["details"]
                label = label[0]
                # removes unwanted labels and null classes
                if label["value"] == 'null' or label["value"] == remveLbl:  
                    continue
                else:
                    # removes label
                    if swapLabel:
                        label = switchLabel(swapLabel, label)
                    # changes class label numbers based on the removed value
                    if label["value"] > remveLbl:
                        if remveLbl != -1:
                            label["value"] -= 1
                    
                    
                    # original values to view all the labels
                    labels.append(label["value"])
                    width.append(ann["width"])
                    height.append(ann["height"])
                    xCord.append(ann["x"])
                    yCord.append(ann["y"])
                    IP.append(userIP)
                    if includeIID == True:
                        labelsIID.append(label["value"])
                        widthIID.append(ann["width"])
                        heightIID.append(ann["height"])
                        xCordIID.append(ann["x"])
                        yCordIID.append(ann["y"])
                        IPIID = userIP
            if xCordIID:
                x2, y2, w2, h2 = PrepForYolo(xCordIID,yCordIID,widthIID,heightIID,originalwidth,originalheight)
                x2, y2, w2, h2 = fixOutOfBoundsBoxes(x2, y2, w2, h2)
                formats = pd.DataFrame({'label':labelsIID, 'xCord':x2, 'yCord':y2, 'width':w2, 'height':h2})
                SaveIID(formats, IPIID, saveLoaction, pathTypeIID, name, imageLocation)
                    
            # adds compleeted rows to a list to be skpped later on. To avoid repeating labels.
            indexDone.append(index2)
        if xCord:
            x2, y2, w2, h2 = PrepForYolo(xCord,yCord,width,height,originalwidth,originalheight)
            x2, y2, w2, h2 = fixOutOfBoundsBoxes(x2, y2, w2, h2)
            formats = pd.DataFrame({'label':labels, 'xCord':x2, 'yCord':y2, 'width':w2, 'height':h2})
            SaveCrowd(formats, IP, saveLoaction, pathType, name, imageLocation)
        
        # adds data to a new csv for easy data analysis
        
        

            
# removes repeated images in the train dataset that are already in the test dataset
removeRepeated(testLocationImg, saveLoaction, pathType)
if includeIID == True:
    removeRepeated(testLocationImg, saveLoaction, pathTypeIID)

# performs train val split, and imports test dataset
valSplit(trainValSplit, saveLoaction, pathType, False)
if includeTestDS:
    injectTestDS(testLocationImg, testLocationLbl, saveLoaction, pathType, False)
if includeIID == True:
    valSplit(trainValSplit, saveLoaction, pathTypeIID, True)
    if includeTestDS:
        injectTestDS(testLocationImg, testLocationLbl, saveLoaction, pathTypeIID, True)
print("############ DONE ############")

960it [00:12, 72.67it/s] 

REMOVED UNWANTED IMAGES:  Unknown-X-20200928-091843-XIQ+LFYOKA+S-0-YunpengÔÇÖs iMac.JPG


2486it [00:34, 65.93it/s]

REMOVED UNWANTED IMAGES:  Unknown-X-20200928-091843-XIQ+LFYOKA+S-0-YunpengÔÇÖs iMac.JPG


4510it [01:02, 78.48it/s]

...

REMOVED UNWANTED IMAGES:  Unknown-X-20200928-091843-XIQ+LFYOKA+S-0-YunpengÔÇÖs iMac.JPG


10318it [02:28, 75.92it/s]

REMOVED UNWANTED IMAGES:  Unknown-X-20200928-091843-XIQ+LFYOKA+S-0-YunpengÔÇÖs iMac.JPG


10789it [02:35, 69.26it/s]


79 Repeated Images. They Have Been Deleted: 
['IS20180614_161235_0835_000000CD', 'IS20190121_094120_0903_00000702', 'IS20190529_121316_0603_00001B5F', 'IS20190605_100053_0811_00001CCE', 'IS20190716_144826_0668_000026F2', 'IS20190716_144902_0220_000026F3', 'IS20190916_155531_0888_00003368', 'IS20190925_080311_0732_0000343E', 'IS20190930_080551_0119_000034B9', 'IS20191111_161011_0377_0000387B', 'IS20191113_081626_0893_000038CF', 'IS20191128_142111_0361_00003A3B', 'IX20191008_105239_0006_00003581', 'IX20191018_145924_0858_000036ED', 'IX20191022_101757_0873_00003708', 'IX20191023_142706_0883_0000373E', 'IX20191024_142538_0552_00003761', 'IX20191030_093817_0791_000037B5', 'Unknown-X-20191111-111540-XE2TAIKRCCFW-3', 'Unknown-X-20191113-094338-X9SZC0TUBDEN-3', 'Unknown-X-20191113-111853-XYYUCGZJHAAT-3', 'Unknown-X-20191114-085826-X4MKCC4Y2IX4-3', 'Unknown-X-20191115-092306-XEMAYUBISHR7-3', 'Unknown-X-20191119-084034-XKT545WVTDVB-3', 'Unknown-X-20191119-090744-X99LU4EIHRX1-3', 'Unknown-X-20191